In [ ]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import nn
import collections
from rdkit.Chem import Descriptors
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix


In [ ]:
data = pd.read_csv('solvent_dataset.csv', sep="\t",low_memory=False)

In [ ]:
#params
#for quick prototyping epoch count is set to 20
epoch_count = 20

In [ ]:
# Vectorize input 
smiles_train = data["RP"]
print (smiles_train.shape)
charset = set("".join(list(data.RP))+"!E")
char_to_int = dict((c,i+1) for i,c in enumerate(charset))
int_to_char = dict((i+1,c) for i,c in enumerate(charset))
embed = max([len(smile) for smile in data .RP])
print (str(charset))
print(len(charset), embed)

def vectorize(smiles):
        one_hot =  np.zeros((smiles.shape[0], embed ),dtype=np.int8)
        print(one_hot.shape)
        for i,smile in enumerate(smiles):
            #encode the startchar
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j] = char_to_int[c]
            #Encode endchar
            
        #Return two, one for input and the other for output
        return one_hot#, one_hot[:,0:-1,:]
X_train = vectorize(smiles_train.values)

In [ ]:
def get_best_s():
    zest = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    zesp = 0.2
    zel = []
    maxpr = 0
    best_nr = 0
    bad_count = 0
    for zzyz in range(0,100):

        try:
            preds1 = modelx.predict(X_test2,verbose=0,batch_size=512)
            #print(preds1)
            #print(y_test2)
            preds1[preds1>=zesp] = 1
            preds1[preds1<1-zesp] = 0
            klkl = accuracy_score(y_test2_2, preds1)

            if klkl > maxpr:
                maxpr = klkl
                best_nr = zesp
            #print (klkl)
        except:
            bad_count +=1
        if bad_count >4:
            break
        zesp+=0.005
        zel.append(klkl)
    #print(best_nr)
    #plt.plot(zel)
    print(maxpr)
    #print(np.array(zel).max())
    #plt.plot(zel)
    return best_nr

In [ ]:
# vectorize output
charset_t = collections.Counter((" ".join(list(data.crude))).split(' '))
#charset_t = collections.Counter(list(data.crude))
print(charset_t)
charset_t=set(charset_t)
char_to_int_t = dict((c,i) for i,c in enumerate(charset_t))
int_to_char_t = dict((i,c) for i,c in enumerate(charset_t))
print(char_to_int_t)
embed_t =  len(charset_t)
print(embed_t)
one_hot_y = np.zeros((len(data), (len(charset_t)) ),dtype=np.int8)
len(one_hot_y[0])
for i,nam in enumerate(data['crude']):
    list_short = list(nam.split(" "))
    first = True
    for z, name in enumerate(list_short):
        if first == True:
            one_hot_y[i][char_to_int_t[name]] = 1
            #first = False
        #elif first !=True:
        #    one_hot_y[i][char_to_int_t[name]+11] = 1
            
Y_train = np.array(one_hot_y)
 

In [ ]:
embed_smiles = len(charset) +1
input_shape = X_train.shape[1]

In [ ]:
X_train = np.array(X_train)
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
inputA = keras.Input(shape=(199,),name='inputx')


x = layers.Embedding(embed_smiles,12, input_length=199)(inputA)


x = layers.Dense(128, activation="relu")(x)

x = layers.Dense(32, activation="relu")(x)

x = layers.Flatten()(x)

p = layers.Dense(64, activation='relu')(x)

p = layers.Dense(10, activation="sigmoid",dtype='float32',name="classification")(p)


modelx = keras.Model(inputs=inputA, outputs=p)

modelx.compile(loss='binary_crossentropy', optimizer="adam", metrics='binary_accuracy')
print (modelx.summary())

In [ ]:
history = modelx.fit(X_train2,y_train2_2, epochs=epoch_count, batch_size=256, validation_split = 0.1)

In [ ]:
preds1 = modelx.predict(X_test2,verbose=1,batch_size=512)
#print(preds1)
#print(y_test2)
ac_sc = get_best_s()
preds1[preds1>=ac_sc] = 1
preds1[preds1<1-ac_sc] = 0
print (accuracy_score(y_test2_2, preds1))
print (classification_report(y_test2_2, preds1))
print (multilabel_confusion_matrix(y_test2_2, preds1))

In [ ]:
inputA = keras.Input(shape=(199,),name='inputx')


x = layers.Embedding(embed_smiles,12, input_length=199)(inputA)

x = layers.Permute((2,1))(x)

x = layers.LSTM(128,return_sequences=False)(x)

p = layers.Dense(128, activation='relu')(x)

p = layers.Dense(64, activation='relu')(x)

p = layers.Dense(10, activation="sigmoid",dtype='float32',name="classification")(p)

modelx = keras.Model(inputs=inputA, outputs=p)

modelx.compile(loss='binary_crossentropy', optimizer="adam", metrics='binary_accuracy')
print (modelx.summary())

In [ ]:
history = modelx.fit(X_train2,y_train2_2, epochs=epoch_count, batch_size=256, validation_split = 0.1)

In [ ]:
preds1 = modelx.predict(X_test2,verbose=1,batch_size=512)
#print(preds1)
#print(y_test2)
ac_sc = get_best_s()
preds1[preds1>=ac_sc] = 1
preds1[preds1<1-ac_sc] = 0
print (accuracy_score(y_test2_2, preds1))
print (classification_report(y_test2_2, preds1))
print (multilabel_confusion_matrix(y_test2_2, preds1))

In [ ]:
inputA = keras.Input(shape=(199,),name='inputx')


x = layers.Embedding(embed_smiles,12, input_length=199)(inputA)



k = layers.Conv1D(64,12,activation='relu')(x)
k = layers.AveragePooling1D(3)(k)
k = layers.Conv1D(64,12,activation='relu')(k)
k = layers.AveragePooling1D(3)(k)
k = layers.Conv1D(64,5,activation='relu')(k)
k = layers.AveragePooling1D(3)(k)

k = layers.Flatten()(k)

p = layers.Dense(64, activation='relu')(k)

#p = layers.Dense(64, activation='relu')(p)

p = layers.Dense(10, activation="sigmoid",dtype='float32',name="ratio")(p)

modelx = keras.Model(inputs=inputA, outputs=p)

modelx.compile(loss='binary_crossentropy', optimizer="adam", metrics='binary_accuracy')
modelx.summary()

In [ ]:
history = modelx.fit(X_train2,y_train2_2, epochs=epoch_count, batch_size=256, validation_split = 0.1)

In [ ]:
preds1 = modelx.predict(X_test2,verbose=1,batch_size=512)
#print(preds1)
#print(y_test2)
ac_sc = get_best_s()
preds1[preds1>=ac_sc] = 1
preds1[preds1<1-ac_sc] = 0
print (accuracy_score(y_test2_2, preds1))
print (classification_report(y_test2_2, preds1))
print (multilabel_confusion_matrix(y_test2_2, preds1))